# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gvts
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tocopilla,-22.0920,-70.1979,18.31,67,9,2.93,CL,1724791313
1,1,mount gambier,-37.8333,140.7667,10.14,62,74,7.60,AU,1724791323
2,2,fortuna,40.5982,-124.1573,20.19,76,0,6.17,US,1724791326
3,3,kisangani,0.5167,25.2000,23.31,84,53,0.95,CD,1724791328
4,4,yellowknife,62.4560,-114.3525,20.23,76,75,2.57,CA,1724791330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng",
                                          "Lat",
                                          geo=True,
                                          tiles="OSM",
                                          frame_width=600,
                                          frame_height=400,
                                          size="Humidity",
                                          scale=0.1,
                                          color="City")


# Display the map
humidity_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_city = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <=30)]

# Drop any rows with null values
criteria_city = criteria_city.dropna()

# Display sample data
criteria_city=criteria_city.sample(n=20)
criteria_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
408,408,san julian,21.0167,-102.1667,25.26,41,88,0.16,MX,1724791909
520,520,carboneras,37.0015,-1.9213,25.86,75,2,1.21,ES,1724792163
332,332,at taj,24.2049,23.2857,29.94,41,34,7.38,LY,1724791870
149,149,palmer,42.1584,-72.3287,29.06,53,40,2.06,US,1724791622
47,47,brasilia,-15.7797,-47.9297,28.51,30,4,3.09,BR,1724791171
192,192,kuqa,41.7278,82.9364,22.95,44,8,2.84,CN,1724791679
251,251,phan rang-thap cham,11.5667,108.9833,25.75,79,98,1.22,VN,1724791760
71,71,souillac,-20.5167,57.5167,21.21,78,75,8.75,MU,1724791512
278,278,dongargarh,21.2000,80.7333,23.50,93,95,2.18,IN,1724791798
62,62,constantia,44.1833,28.6500,23.38,69,100,7.34,RO,1724791499


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns= ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = criteria_city[columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
408,san julian,MX,21.0167,-102.1667,41,
520,carboneras,ES,37.0015,-1.9213,75,
332,at taj,LY,24.2049,23.2857,41,
149,palmer,US,42.1584,-72.3287,53,
47,brasilia,BR,-15.7797,-47.9297,30,
192,kuqa,CN,41.7278,82.9364,44,
251,phan rang-thap cham,VN,11.5667,108.9833,79,
71,souillac,MU,-20.5167,57.5167,78,
278,dongargarh,IN,21.2000,80.7333,93,
62,constantia,RO,44.1833,28.6500,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = f"accommodation.hotel"



params = {"radius":radius,
"apiKey":geoapify_key,
"categories":categories,
"limit":limit}
            

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san julian - nearest hotel: No hotel found
carboneras - nearest hotel: CaSabah
at taj - nearest hotel: No hotel found
palmer - nearest hotel: No hotel found
brasilia - nearest hotel: Hotel de Trânsito
kuqa - nearest hotel: Kuqa Hotel - foreigners accepted
phan rang-thap cham - nearest hotel: Khách Sạn Ninh Thuận
souillac - nearest hotel: Hotel SkylineInn
dongargarh - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Dali
jacmel - nearest hotel: Hotel L'Amenitie
west island - nearest hotel: Cocos Village Bungalows
enewetak - nearest hotel: No hotel found
ponta delgada - nearest hotel: Hotel Talisman
trincomalee - nearest hotel: Summer set inn
fortaleza - nearest hotel: Hotel Catedral
westport - nearest hotel: Norwalk Inn and Conference Center
port hedland - nearest hotel: The Esplanade Hotel
pierre - nearest hotel: Quality Inn
hasaki - nearest hotel: Choshi Plaza Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
408,san julian,MX,21.0167,-102.1667,41,No hotel found
520,carboneras,ES,37.0015,-1.9213,75,CaSabah
332,at taj,LY,24.2049,23.2857,41,No hotel found
149,palmer,US,42.1584,-72.3287,53,No hotel found
47,brasilia,BR,-15.7797,-47.9297,30,Hotel de Trânsito
192,kuqa,CN,41.7278,82.9364,44,Kuqa Hotel - foreigners accepted
251,phan rang-thap cham,VN,11.5667,108.9833,79,Khách Sạn Ninh Thuận
71,souillac,MU,-20.5167,57.5167,78,Hotel SkylineInn
278,dongargarh,IN,21.2000,80.7333,93,No hotel found
62,constantia,RO,44.1833,28.6500,69,Hotel Dali


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat",  
                                   geo=True, 
                                   tiles="OSM", 
                                   frame_width=600, 
                                   frame_height=400, 
                                   size="Humidity", 
                                   scale=0.8, 
                                   color="City",
                                   hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)